
## Time series analysis (trend, slope, ... calculation) for JEDI-CUBES
This notebook was set up to help with the analysis of time series in jedi cubes. It can be connected directly to JEDI cubes and examine a time series contained there for various reference units (NUTS3,2,1,0, 1km, 10km, ....). If you have any questions, please contact: loehnertz@space4environment.com




(1) connection to jedi cube tables on MS-SQL (TEAL and GREENMONKEY)
!Please note: you need read and partially write permissions for the database to be used.!

Hallo Eva

In [12]:
# Reading libaries and start a test-connection to different MS-SQL Server:
import pyodbc 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy as sa
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL
import json

from scipy import stats

#check processing time:##
import time  
import datetime
start_time = time.time()
##########################

#### GREENMONKEY

print ("connect to engine GREENMONKEY")
engine_GREENMONKEY = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
connection = engine_GREENMONKEY.raw_connection()
cursor = connection.cursor()
print ("connection to greenmonkey found")
print ("the function HAS_DBACCESS returns 1 if the user has access to the database, otherwise, it will return as 0")  
#


query_check=('''       
SELECT [admin_category]
FROM [drought].[SMA_GDMP_cube_2]
      ''')
print (query_check)
readable_database = pd.read_sql(query_check, engine_GREENMONKEY)
cursor.execute(query_check)
cursor.close()
##connection.commit()
#print ("following databases can be READ:")
##print (readable_database)
#print ("----------------------------------------------------------------")
#
#
#
#

print ("end....")
print("--- %s seconds ---" % (time.time() - start_time))
seconds = time.time() - start_time
print('Time Taken: HH:MM:SS ', time.strftime("%H:%M:%S",time.gmtime(seconds)))

connect to engine GREENMONKEY
connection to greenmonkey found
the function HAS_DBACCESS returns 1 if the user has access to the database, otherwise, it will return as 0
       
SELECT [admin_category]
FROM [drought].[SMA_GDMP_cube_2]
      
end....
--- 2.2699880599975586 seconds ---
Time Taken: HH:MM:SS  00:00:02


### (1) reading CUBE 2 

[Climate_Impact].[drought].[SMA_GDMP_cube_2]

In [13]:
## connect to MS-sql  



## testing connection with simple query:
print ("Reading drought table from greenmonkey")
query_drought_cube2=('''
SELECT  
    [admin_category]
      ,[GridNum10km]
      ,[LULUCF_CODE]
      ,[env_zones]
      ,[natura2000_protection]
      ,[Areaha]
     
      ,[annual_drought_productivity_deficit_2000]
      ,[annual_drought_productivity_deficit_2001]
      ,[annual_drought_productivity_deficit_2002]
      ,[annual_drought_productivity_deficit_2003]
      ,[annual_drought_productivity_deficit_2004]
      ,[annual_drought_productivity_deficit_2005]
      ,[annual_drought_productivity_deficit_2006]
      ,[annual_drought_productivity_deficit_2007]
      ,[annual_drought_productivity_deficit_2008]
      ,[annual_drought_productivity_deficit_2009]
      ,[annual_drought_productivity_deficit_2010]
      ,[annual_drought_productivity_deficit_2011]
      ,[annual_drought_productivity_deficit_2012]
      ,[annual_drought_productivity_deficit_2013]
      ,[annual_drought_productivity_deficit_2014]
      ,[annual_drought_productivity_deficit_2015]
      ,[annual_drought_productivity_deficit_2016]
      ,[annual_drought_productivity_deficit_2017]
      ,[annual_drought_productivity_deficit_2018]
      ,[annual_drought_productivity_deficit_2019]
      ,[annual_drought_productivity_deficit_2020]
      ,[annual_drought_productivity_deficit_2021]
      ,[annual_drought_productivity_deficit_2022]
  FROM [Climate_Impact].[drought].[SMA_GDMP_cube_2]


''')

df_drought_cube2 = pd.read_sql(query_drought_cube2, engine_GREENMONKEY) ###### import table into a dataframe (df) for the use in pandas!!!!!!!!!!!



print (" sql table as data frame for the next steps:")
print (df_drought_cube2)



## close connection:
#cursor.close(
#connection.commit()




print ("reading cube 2 done")

Reading drought table from greenmonkey
 sql table as data frame for the next steps:
        admin_category   GridNum10km LULUCF_CODE env_zones  \
0                  750  9.439333e+15          CL       MDS   
1                 1439  9.439874e+15          FL       MDS   
2                  365  9.454095e+15          FL       MDM   
3                  366  9.454842e+15          CL       MDS   
4                  366  9.455078e+15          FL       MDS   
...                ...           ...         ...       ...   
542742             664  1.917913e+16          GL       ALN   
542743             323  2.286153e+16          SL       MDM   
542744            1032  2.319031e+16          GL       PAN   
542745             771  2.321078e+16          GL       PAN   
542746             731  2.338223e+16          SL       CON   

              natura2000_protection  Areaha  \
0       none Nature 2000 protection  1931.0   
1                        Natura2000   854.0   
2                        Natur

In [14]:
df_annual_drought_productivity_deficit = df_drought_cube2

print ("table read from TEAL job done")
## Tronsforme the table columns to rows:
df_transformed =df_annual_drought_productivity_deficit.melt(id_vars=['admin_category', 'GridNum10km', 'LULUCF_CODE', 'env_zones', 'natura2000_protection','Areaha'], 
     var_name="year_string", 
       value_name="annual_drought_productivity_deficit")

print ("table TRANSFORMED -pivot- job done")


#Convert the Strings to Datetime in the DataFrame
count_row = df_transformed.shape[0]  # Gives number of rows
count_col = df_transformed.shape[1]  # Gives number of columns#

print ('number of rows: ' + str(count_row))
print ('number of columns: ' + str(count_col))


#cleaning the table: change the year from string to INT or DATE-format:
df_transformed['year_string_number'] = df_transformed['year_string'].str[-4:]
df_transformed['year'] = df_transformed['year_string_number'].astype(int)
df_transformed.drop(['year_string_number'], axis=1)
#df_transformed['year'] = pd.to_datetime(df_transformed['year'], format='%Y')
print ("year value updated done")


######################################## calcualtion slope,...
# 
# ## group ############################################################### =['Admin', 'Env', 'CLC00', 'CLC18','AreaHa']
# 
# ## here we are doing a groupby -(in the moment we have also lines for CLC06, CL12,) --> group by can also be done during SQL reading
dfg = df_transformed.groupby(["admin_category","GridNum10km","LULUCF_CODE","env_zones","natura2000_protection","year"]).agg({'Areaha' : ['sum'], 'annual_drought_productivity_deficit' : ['mean']}).reset_index()  # this function do a group by and calc. the sum area and min,max,avg for LINT
# #cleaning columns:
dfg.columns = ["_".join(x) for x in dfg.columns.ravel()]
dfg = dfg.rename(columns={'admin_category_': 'admin_category', 'GridNum10km_': 'GridNum10km', 'LULUCF_CODE_': 'LULUCF_CODE', 'env_zones_': 'env_zones', 'natura2000_protection_': 'natura2000_protection','year_': 'year'}).reset_index()#.sort_values(ascending=True)
# # calculate weighted AVG:
# #dfg['LINT_anom_values_weightavg'] =dfg['AreaHa_sum'] * dfg['LINT_anom_values_mean']   ## not needed for C-tables 
# 
# 
# ################################################################################# model start
# # model for calculation trend:  https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
def model_trend_stat(row):
   y = row.annual_drought_productivity_deficit_mean
   x = row.year
   count_rows = len(row)  ## better to work with index (loc).but for the moment ok
   year_min=min(x)
   year_max=max(x)
   
   ##testing index...but not working...
   #y_start = row['LINT_anom_values_mean'][row['LINT_anom_values_mean']  == year_min].index.tolist()
   #y_start =dfg.loc[row['LINT_anom_values_mean'] == 2]
   #y_end = row.loc[row['LINT_anom_values_mean'] == year_max]  #.index[0]
                       
   y_end = row['annual_drought_productivity_deficit_mean'].values[count_rows-1]  # get the las value in the group
   y_start = row['annual_drought_productivity_deficit_mean'].values[0]  # get the first value in the group    
   
   #slope_change = (y_end - y_start) / y_start  ### cehck divide by zero
   if y_start != 0: 
       slope_change = (y_end - y_start) / y_start
   else:
       slope_change = 0
   
   m = stats.linregress(x,y)
   slope =m.slope 
   intercept=m.intercept
   rvalue=m.rvalue
   pvalue=m.pvalue
   stderr=m.stderr
   intercept_stderr=m.intercept_stderr
   t = [m.slope * i + m.intercept for i in x]  
   #return pd.Series([rvalue,slope,intercept,pvalue,stderr])
   return pd.Series({'rvalue': rvalue, 'slope':slope,'intercept':intercept,'pvalue':pvalue,'stderr':stderr, 'yearMIN': year_min,'yearMAX': year_max,'y_start': y_start,'y_end': y_end,'slope_change': slope_change})
################################################################################ model end
# admin_category","GridNum10km","LULUCF_CODE","env_zones","natura2000_protection"
# #calcualtion of slope, trend,... for every group : (reference unit):
dfg_trend= dfg.groupby(["admin_category", "GridNum10km","LULUCF_CODE","env_zones","natura2000_protection"]).apply(model_trend_stat).reset_index()#.sort_values(ascending=False)

##dfg_trend= dfg.groupby(["Admin", "Env","CLC00","CLC18"]).apply(model_trend_stat).reset_index()#.sort_values(ascending=False)

#print (df_transformed)
print("--- %s seconds ---" % (time.time() - start_time))
seconds = time.time() - start_time
print('Time Taken: HH:MM:SS ', time.strftime("%H:%M:%S",time.gmtime(seconds)))
print ("end....")

table read from TEAL job done
table TRANSFORMED -pivot- job done
number of rows: 12483181
number of columns: 8
year value updated done
--- 463.8808879852295 seconds ---
Time Taken: HH:MM:SS  00:07:43
end....


In [16]:
### update GridNum10km .. set float to INT

dfg_trend['GridNum10km'] = dfg_trend['GridNum10km'].fillna(0).astype(int)
dfg_trend

,admin_category,GridNum10km,LULUCF_CODE,env_zones,natura2000_protection,rvalue,slope,intercept,pvalue,stderr,yearMIN,yearMAX,y_start,y_end,slope_change
0,0,-2147483648,CL,MDN,Natura2000,0.353553,150.350628,-302956.516219,0.097929,86.804976,2000.0,2022.0,-13832.257812,0.0,-1.0
1,0,-2147483648,FL,MDN,Natura2000,0.359519,407.801357,-821847.310707,0.092005,230.973789,2000.0,2022.0,-36991.460938,0.0,-1.0
2,0,-2147483648,GL,MDN,Natura2000,0.353553,60.153872,-121210.052649,0.097929,34.729854,2000.0,2022.0,-5534.156250,0.0,-1.0
3,0,-2147483648,CL,MDN,Natura2000,0.169799,78.867522,-159794.697802,0.438596,99.884893,2000.0,2022.0,-9668.066162,0.0,-1.0
4,0,-2147483648,FL,MDN,Natura2000,0.318023,226.853922,-457662.066799,0.139184,147.579070,2000.0,2022.0,-22397.994141,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542742,1607,-2147483648,none,none,none Nature 2000 protection,0.000000,0.000000,0.000000,1.000000,0.000000,2000.0,2022.0,0.000000,0.0,0.0
542743,1607,-2147483648,none,none,none Nature 2000 protection,0.000000,0.000000,0.000000,1.000000,0.000000,2000.0,2022.0,0.000000,0.0,0.0
542744,1607,-2147483648,none,none,none Nature 2000 protection,0.000000,0.000000,0.000000,1.000000,0.000000,2000.0,2022.0,0.000000,0.0,0.0
542745,1607,-2147483648,none,none,none Nature 2000 protection,0.000000,0.000000,0.000000,1.000000,0.000000,2000.0,2022.0,0.000000,0.0,0.0


Export data frame to greenmonkey:


In [17]:
# #exporting data to GREENMONKEY#################################################################################
print ("connect to engine GREENMONKEY")  #drought
engine_GREENMONKEY = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Climate_Impact" + '?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')
name_of_table ="Drought_trend_xxxxxxxx"       
print ("connect to greenmonkey engine.....GREENMONKEY and store sql table")  
dfg_trend.to_sql(name_of_table, engine_GREENMONKEY, if_exists='replace', index = False, schema='drought')
# 
# ##################################################################################################################

connect to engine GREENMONKEY
connect to greenmonkey engine.....GREENMONKEY and store sql table


91